In [1]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, ImageOps,Image,ImageTk
import numpy as np


In [2]:
# Load your pre-trained model
model = load_model("mnist.h5")


def predict_digit(img):
    # Resize image to 28x28 pixels
    img = img.resize((28, 28))
    # Convert RGB to grayscale
    img = img.convert("L")
    img = np.array(img)
    # Reshape to support our model input and normalize
    img = img.reshape(1, 28, 28, 1)
    img = img / 255.0
    # Predicting the class
    res = model.predict(img)[0]
    return np.argmax(res), max(res)


class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0

        # Creating elements
        # mnist dataset has images with a black background and text written in white
        self.canvas = tk.Canvas(self, width=300, height=300, bg="white", cursor="cross")
        self.label = tk.Label(self, text="Draw..", font=("Helvetica", 48), fg="black")
        self.classify_btn = tk.Button(
            self, text="Recognize", command=self.classify_handwriting
        )
        self.button_clear = tk.Button(self, text="Clear", command=self.clear_all)

        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=tk.W)
        self.label.grid(row=0, column=1, pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)

        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")

    def classify_handwriting(self):
        x = (
            self.winfo_x() + self.canvas.winfo_x()
        )  # Get the X coordinate relative to the screen
        y = (
            self.winfo_y() + self.canvas.winfo_y()
        )  # Get the Y coordinate relative to the screen
        rect = (x, y, x + self.canvas.winfo_width(), y + self.canvas.winfo_height())
        im = ImageGrab.grab(rect)

        # Invert the colors of the image
        im = ImageOps.invert(im)

        digit, acc = predict_digit(im)
        self.label.configure(text=str(digit) + ", " + str(int(acc * 100)) + "%")

        # Convert the inverted image to a PhotoImage
        img_tk = ImageTk.PhotoImage(im)

        # Display the inverted image in a new window
        img_window = tk.Toplevel(self)
        img_window.title("Inverted Image")
        img_label = tk.Label(img_window, image=img_tk)
        img_label.pack()
        img_window.mainloop()

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r = 8
        self.canvas.create_oval(
            self.x - r, self.y - r, self.x + r, self.y + r, fill="black"
        )


app = App()
app.mainloop()

1/1 [==============================] - 0s 15ms/step
